In [2]:
import pandas as pd

In [3]:
npr = pd.read_csv('npr.csv')
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [5]:
cv =CountVectorizer(max_df=0.95 , min_df=2 ,stop_words='english' )

In [6]:
dtm = cv.fit_transform(npr.Article)

In [7]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## LDA

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

In [9]:
#n component = no of topics
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [10]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Showing Stored Words
## grab the vocabulary words

In [12]:
len(cv.get_feature_names())

54777

In [13]:
type(cv.get_feature_names())

list

In [14]:
import random

In [18]:
for i in range(10):
    random_word_id = random.randint(0,54777)
    print(cv.get_feature_names()[random_word_id])

censorious
kidneys
yodeling
guilty
conspirators
belafonte
progressives
vp
constitution
botswana


### Showing Top Words Per Topic

### grab topics

In [20]:
len(LDA.components_)

7

In [21]:
type(LDA.components_)

numpy.ndarray

In [22]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [23]:
single_topic = LDA.components_[0]

In [24]:
single_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993], dtype=int64)

In [26]:
# Word least representative of this topic
single_topic[18302]

0.14285714309286987

In [27]:
# Word most representative of this topic
single_topic[42993]

6247.245510521082

In [25]:
single_topic.argsort()[-10:]

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993], dtype=int64)

In [28]:
top_word_indices = single_topic.argsort()[-10:]

In [29]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

new
percent
government
company
million
care
people
health
said
says


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [45]:
for index, topic in enumerate(LDA.components_):
    print(f"the top 15 word of topic #{index +1} are")
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

the top 15 word of topic #1 are
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


the top 15 word of topic #2 are
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


the top 15 word of topic #3 are
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


the top 15 word of topic #4 are
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


the top 15 word of topic #5 are
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


the top 15 word of topic #6 are
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'kno

### Attaching Discovered Topic Labels to Original Articles

In [35]:
topic_results = LDA.transform(dtm)

In [37]:
topic_results[0]

array([1.61040465e-02, 6.83341493e-01, 2.25376318e-04, 2.25369288e-04,
       2.99652737e-01, 2.25479379e-04, 2.25497980e-04])

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [39]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [40]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 0], dtype=int64)

In [41]:
npr['topic'] = [topic + 1 for topic in topic_results.argmax(axis=1)]

In [44]:
npr.head()

,Article,topic
0,"In the Washington of 2016, even when the polic...",2
1,Donald Trump has used Twitter — his prefe...,2
2,Donald Trump is unabashedly praising Russian...,2
3,"Updated at 2:50 p. m. ET, Russian President Vl...",2
4,"From photography, illustration and video, to d...",3
